In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94&oquery=%EC%98%81%ED%99%94+%EC%95%84%EB%B0%94%ED%83%80&tqi=iaX7Llp0J1ZssSx9h9ZssssstjZ-258685"
driver.get(url)
time.sleep(1)

In [5]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")
best_movie['영화명']

0              명량
1            극한직업
2       신과함께-죄와 벌
3            국제시장
4      어벤져스: 엔드게임
          ...    
196        비긴 어게인
197            태풍
198            킹콩
199        부러진 화살
200           NaN
Name: 영화명, Length: 201, dtype: object

In [6]:
movie_name = best_movie['영화명'][0]
keyword = "영화 "+ movie_name +" 평점"
keyword

'영화 명량 평점'

In [7]:
# 검색어 입력 및 검색
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
time.sleep(2)

In [ ]:
#main_pack > div.sc_new.cs_common_module.case_empasis.color_12._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_rating > div > div:nth-child(2) > div.cm_filter_area._root._sorting_select_order > div > div > div > div > ul > li:nth-child(1) > a

In [14]:
# 공감순 선택
driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
time.sleep(1)

In [15]:
# 리뷰 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()

# 10번
for i in range(0,10):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [17]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [18]:
제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
제목

'명량'

In [19]:
전체 = soup.select_one('span.area_star_number').text
참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
남자 = soup.select_one('div.area_card_male span.area_star_number').text
여자 = soup.select_one('div.area_card_female span.area_star_number').text
print(전체, 참여자수, 남자, 여자)

8.88 28,765 8.82 8.94


In [20]:
_10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
_20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
_30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
_40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
_50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text
print(_10대, _20대, _30대, _40대, _50대)

9.18 8.92 8.76 8.84 8.82


In [29]:
lis = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
li= lis[3]
관람객별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
관람객리뷰 = li.select_one('div.area_text_expand span.desc._text').text.strip()
print(관람객별점, 관람객리뷰)

10 작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로 400년전 구국의영웅이 2014 우리에게 벅찬 울림을 전한다


In [27]:
# 평론가 클릭 및 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()

for i in range(0,10):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [28]:
lis = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
li= lis[1]
평론가별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
평론가리뷰 = li.select_one('div.area_text_expand').text.strip()
print(평론가별점, 평론가리뷰)

6.75 수식 없는 묵직한 정공법으로 끝까지 밀어붙인다. 비장하고 진지하고 숙연하다. 해전 신의 놀라운 밀도에 비해 이순신을 제외한 캐릭터들의 아쉬운 배치와 활용은 극의 밀도를 떨어뜨린다. 펼쳐보기


In [ ]:
#main_pack > div.sc_new.cs_common_module.case_empasis.color_10._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_info > div > div.detail_info > dl

In [8]:
# 기본정보 클릭 러닝타임 추출
driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
time.sleep(2)

divs = driver.find_elements(By.CSS_SELECTOR,'div.cm_content_area._cm_content_area_info div.detail_info dl div')
if '러닝타임' in div.text:
    러닝타임 = div.text.split()[1]

In [57]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")

df1 = []
df2 = []
df3 = []

# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94&oquery=%EC%98%81%ED%99%94+%EC%95%84%EB%B0%94%ED%83%80&tqi=iaX7Llp0J1ZssSx9h9ZssssstjZ-258685"
driver.get(url)
time.sleep(1)

for movie_name in best_movie['영화명']:
    
    try: 
        # 검색어 설정
        keyword = "영화 "+ str(movie_name) +" 평점"

        # 검색어 입력 및 검색
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
        driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
        time.sleep(3)

        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)


        # 관람객 리뷰 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)

        for i in range(0,3):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        # 평론가 클릭 및 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
        driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
        time.sleep(1)

        for i in range(0,3):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
        전체 = soup.select_one('span.area_star_number').text
        참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
        남자 = soup.select_one('div.area_card_male span.area_star_number').text
        여자 = soup.select_one('div.area_card_female span.area_star_number').text

        _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
        _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
        _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
        _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
        _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

        # 관람객 리뷰
        lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')

        for li1 in lis1:
            관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
            df2.append([movie_name, 제목, 관람객별점, 관람객리뷰])

        # 평론가 리뷰
        lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')

        for li2 in lis2:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df3.append([movie_name, 제목, 평론가별점, 평론가리뷰])

        # 기본정보 클릭 러닝타임 추출
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(2)

   
        sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
        divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
        러닝타임 = '-'

        for div in divs:
            if '러닝타임' in div.text:
                러닝타임 = div.text.split()[1]
            elif '개봉' in div.text:
                개봉 = div.text.split()[1] 

        df1.append([movie_name, 제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대])   
    
    except:
        print(movie_name)
        
star = pd.DataFrame(df1, columns=['movie_name', '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
review = pd.DataFrame(df2, columns=['movie_name', '제목', '관람객별점', '관람객리뷰'])
critic_review = pd.DataFrame(df3, columns=['movie_name', '제목', '평론가별점', '평론가리뷰'])

겨울왕국
엑시트
디워
트랜스포머
캡틴 마블
캐리비안의 해적: 세상의 끝에서
가문의 위기(가문의 영광2)
아이언맨
박물관이 살아있다!
인디아나 존스 4 : 크리스탈 해골의 왕국
댄싱퀸
캐리비안의 해적 : 망자의 함
쌍화점
귀향
nan


In [58]:
star

,movie_name,제목,러닝타임,전체,참여자수,남자,여자,_10대,_20대,_30대,_40대,_50대
0,명량,명량,128분,8.88,"28,765",8.82,8.94,9.18,8.92,8.76,8.84,8.82
1,극한직업,극한직업,111분,9.20,"15,943",9.16,9.24,9.43,9.32,9.22,9.09,8.82
2,신과함께-죄와 벌,신과함께-죄와 벌,139분,8.73,"26,216",8.71,8.75,9.09,8.76,8.66,8.73,8.44
3,국제시장,국제시장,126분,9.16,"20,296",9.21,9.11,9.41,9.20,9.09,9.11,9.14
4,어벤져스: 엔드게임,어벤져스: 엔드게임,181분,9.50,"31,975",9.52,9.45,9.81,9.65,9.48,9.16,8.89
...,...,...,...,...,...,...,...,...,...,...,...,...
181,해리 포터와 불의 잔,해리 포터와 불의 잔,156분,9.41,128,9.00,9.50,9.58,9.39,9.38,9.24,9.75
182,비긴 어게인,비긴 어게인,104분,9.13,"7,381",9.18,9.09,9.41,9.22,8.98,8.79,8.89
183,태풍,태풍,124분,6.13,"6,934",5.63,6.98,6.52,5.97,5.95,6.41,5.25
184,킹콩,킹콩,186분,8.97,"9,200",9.01,8.91,8.91,9.05,8.84,8.99,9.24


In [63]:
star['movie_name'].nunique()

186

In [64]:
review

,movie_name,제목,관람객별점,관람객리뷰
0,명량,명량,10,오늘 아침에 조조로 보고왔습니다. 8월4일에 해군에 입대하는데. 그전에 보기 잘한 ...
1,명량,명량,9,"극찬할 수는 없지만 이 정도면 내용, 영상, 연출 전부 꽤나 뛰어난 작품이지 않은가..."
2,명량,명량,10,보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화ㅠㅠ한 순간도 놓칠 수 없는 해전씬이...
3,명량,명량,10,작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 ...
4,명량,명량,8,전쟁씬에서 입벌리고봄
...,...,...,...,...
7293,부러진 화살,부러진 화살,10,대한민국이 이런나라였구나..좌절감이 많이 듭니다
7294,부러진 화살,부러진 화살,10,사법부 결국 국민앞에 무너진다
7295,부러진 화살,부러진 화살,8,법은 없고 판사들의 동료애만 있다. 의료사고 나면 의사들끼리 편 먹듯이 가재는 게편...
7296,부러진 화살,부러진 화살,10,대한민국 사법부 썩을대로 썩었다


In [65]:
review['제목'].value_counts()

반도                 80
어벤져스: 엔드게임         80
어메이징 스파이더맨 2       80
아바타: 물의 길          80
스파이더맨: 노 웨이 홈      80
                   ..
더 테러 라이브: 스쿨 어택    22
다크 나이트 라이즈         16
독전: 익스텐디드 컷        10
검은 사제들: 지옥의 문      10
비긴 어게인             10
Name: 제목, Length: 177, dtype: int64

In [66]:
review[review['제목']=="스즈메의 문단속"]

,movie_name,제목,관람객별점,관람객리뷰
3688,스즈메의 문단속,스즈메의 문단속,6,<너의 이름은>에 이어 동일본대지진에 대한 트라우마를 이 영화를 통해 다시금 다뤄보...
3689,스즈메의 문단속,스즈메의 문단속,6,초반까지는 신선한 설정과 아름다운 작화로 흥미로웠으나 뒤로 갈 수록 이전 작품에서 ...
3690,스즈메의 문단속,스즈메의 문단속,10,감독님의 영화를 좋아하지 않아도 한 번쯤은 볼만합니다. 너의 이름은 이나 날씨의 아...
3691,스즈메의 문단속,스즈메의 문단속,10,마지막 클라이막스때 저항없이 울었다
3692,스즈메의 문단속,스즈메의 문단속,3,금사빠 스즈메의 얼렁뚱땅 이야기 이딴 스토리로 무슨.. 작화와 ost가 아깝습니다.
3693,스즈메의 문단속,스즈메의 문단속,10,폐허가 된 자리에도 마음에도 꽃은 핀다
3694,스즈메의 문단속,스즈메의 문단속,4,"영상미는 좋음 볼거면 아이맥스로 보세요.근데, 전체적인 개연성 너어어어어무 부족. ..."
3695,스즈메의 문단속,스즈메의 문단속,4,보는 내내 고등학생때 강제로 야자 하라고 붙잡혀있는 기분이였음그림은 예뻤으나 진부한...
3696,스즈메의 문단속,스즈메의 문단속,10,너의이름은 날씨의아이에 이어 재난3부작의 정점을 제대로 찍은 수작! 역대급 영상미와...
3697,스즈메의 문단속,스즈메의 문단속,10,"역대급 영상미, 역대급 감동.. 진짜 한참 울었습니다. 재난은 진짜 너무 무서워요.."


In [67]:
critic_review

,movie_name,제목,평론가별점,평론가리뷰
0,명량,명량,7.0,캐릭터에 대한 깊이 있는 고찰이나 당대에 대한 역사의식이나 고증의 완벽함에 대한 기...
1,명량,명량,6.75,수식 없는 묵직한 정공법으로 끝까지 밀어붙인다. 비장하고 진지하고 숙연하다. 해전 ...
2,명량,명량,6.0,해전신은 꽤 볼만하다
3,명량,명량,5.0,인물은 흐릿하고 해전만 요란하다
4,명량,명량,6.0,"해상 전투는 순류, 나머지는 역류. 반반 무(武) 많이"
...,...,...,...,...
1477,부러진 화살,부러진 화살,7.0,"의미와 재미, 한국영화 어른들의 힘!"
1478,부러진 화살,부러진 화살,8.0,사법부의 심장을 향해 겨누어진 실화 한방!
1479,부러진 화살,부러진 화살,7.0,국민배우 안성기 구하기
1480,부러진 화살,부러진 화살,8.0,"아름다워라, 이것이 선배의 영화"


In [37]:
critic_review['제목'].value_counts()

명량           10
극한직업         10
신과함께-죄와 벌    10
Name: 제목, dtype: int64

In [48]:
# # 검색어 설정
# movie_name = "1987"

In [ ]:
#main_pack > div.sc_new.cs_common_module.case_empasis.color_13._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_rating > div > div:nth-child(2) > div.cm_filter_area._root._sorting_select_order > div > div > div > div > ul > li:nth-child(1) > a

In [49]:
# # 검색어 설정
# keyword = "영화 "+ str(movie_name) +" 평점"

# # 검색어 입력 및 검색
# driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
# driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
# driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
# time.sleep(2)

# # 리뷰 공감순 선택
# driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
# time.sleep(1)
# driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
# time.sleep(1)


# # 관람객 리뷰 스크롤 내리기
# driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
# time.sleep(1)

# for i in range(0,3):
#     driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
#     time.sleep(1)

# # 평론가 클릭 및 스크롤 내리기
# driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
# driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
# time.sleep(1)

# for i in range(0,3):
#     driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
#     time.sleep(1)

# soup = BeautifulSoup(driver.page_source, "html.parser")

# 제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
# 전체 = soup.select_one('span.area_star_number').text
# 참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
# 남자 = soup.select_one('div.area_card_male span.area_star_number').text
# 여자 = soup.select_one('div.area_card_female span.area_star_number').text

# _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
# _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
# _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
# _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
# _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

# # 관람객 리뷰
# lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')

# for li1 in lis1:
#     관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
#     관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
#     df2.append([movie_name, 제목, 관람객별점, 관람객리뷰])

# # 평론가 리뷰
# lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')

# for li2 in lis2:
#     평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
#     평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
#     df3.append([movie_name, 제목, 평론가별점, 평론가리뷰])

# # 기본정보 클릭 러닝타임 추출
# driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
# time.sleep(2)


# sub_soup = BeautifulSoup(driver.page_source, "html.parser")

# divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')

# 러닝타임 = '-'

# for div in divs:
#     if '러닝타임' in div.text:
#         러닝타임 = div.text.split()[1]

# df1.append([movie_name, 제목, 러닝타임, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대])  

In [68]:
star.to_excel('[크롤링] 역대_네이버_영화_평점_원본.xlsx', index=False)

In [69]:
review.to_excel('[크롤링] 역대_네이버_영화_관람객_리뷰_원본.xlsx', index=False)

In [71]:
critic_review.to_excel('[크롤링] 역대_네이버_영화_전문가_리뷰_원본.xlsx', index=False)

In [72]:
star.to_csv('[크롤링] 역대_네이버_영화_평점_원본.csv', index=False)

In [73]:
review.to_csv('[크롤링] 역대_네이버_영화_관람객_리뷰_원본.csv', index=False)

In [74]:
critic_review.to_csv('[크롤링] 역대_네이버_영화_전문가_리뷰_원본.csv', index=False)

In [75]:
sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')

[<div class="info_group"> <dt><span class="cm_bar"></span>개봉</dt> <dd>2012.01.18.</dd> </div>,
 <div class="info_group"> <dt class=""><span class="cm_bar"></span>등급</dt> <dd>15세 관람가</dd> </div>,
 <div class="info_group"> <dt><span class="cm_bar"></span>장르</dt> <dd>드라마</dd> </div>,
 <div class="info_group"> <dt><span class="cm_bar"></span>국가</dt> <dd>대한민국</dd> </div>,
 <div class="info_group"> <dt><span class="cm_bar"></span>러닝타임</dt> <dd>100분</dd> </div>,
 <div class="info_group"> <dt><span class="cm_bar"></span>배급</dt> <dd>(주)NEW</dd> </div>]

In [84]:
divs[0].text.split()[1]


'2012.01.18.'

In [85]:
best_movie

,순위,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,국적,배급사
0,1,명량,2014-07-30,135758658810,17615919,1587,188726,한국,한국,(주)씨제이이엔엠
1,2,극한직업,2019-01-23,139655543516,16266338,2003,292816,한국,한국,(주)씨제이이엔엠
2,3,신과함께-죄와 벌,2017-12-20,115727528087,14414658,1912,214631,한국,한국,롯데쇼핑㈜롯데엔터테인먼트
3,4,국제시장,2014-12-17,110947621230,14264478,1044,212705,한국,한국,(주)씨제이이엔엠
4,5,어벤져스: 엔드게임,2019-04-24,122492181020,13977602,2835,246433,미국,미국,월트디즈니컴퍼니코리아 유한책임회사
...,...,...,...,...,...,...,...,...,...,...
196,197,비긴 어게인,2014-08-13,27433947031,3479444,525,87565,미국,미국,판씨네마(주)
197,198,태풍,2005-12-14,22209675300,3471150,481,47155,한국,한국,CJ ENM
198,199,킹콩,2005-12-14,21476688172,3470059,319,38456,미국,미국,UIP코리아
199,200,부러진 화살,2012-01-18,25938096000,3460212,530,71607,한국,한국,(주)넥스트엔터테인먼트월드(NEW)
